# SemEval 2025 Task 9: The Food Hazard Detection Challenge
---

Spyridi Zoi <br>
Department of Management Science and Technology, AUEB<br>
t8210139@aueb.gr<br>
Athens, 2025

---
This notebook presents a machine learning pipeline for detecting food hazards based on textual data. It is designed to address [SemEval 2025 Task 9: The Food Hazard Detection Challenge](https://github.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io). The pipeline includes data preprocessing, feature extraction, model training using Support Vector Machines (SVM), and evaluation of classification performance.

## Dataset Loading

We use two datasets:

* Training Data: Contains labeled food hazard incidents.

* Test Data: Used for evaluating model performance.

The datasets are loaded from external URLs.


In [1]:
import pandas as pd

url_train = "https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/main/data/incidents_train.csv"
url_test = "https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/main/data/incidents_test.csv"

data_train = pd.read_csv(url_train, index_col=0)
data_test = pd.read_csv(url_test, index_col=0)


## Data Preprocessing

Firstly, we need to point out that the text classification will be based on the *title* column as it yields better results.

To prepare the data for improved performance, we follow these steps:

1. Clean the text by removing stopwords, URLs, tags, numbers, punctuation, and special characters.

2. Apply stemming to convert words to their root form (e.g., running → run).

    The text undergoes a cleaning process:

    * Lowercasing: Converts all text to lowercase.

    * Removing URLs, mentions, hashtags, and numbers.

    * Removing punctuation and extra spaces.

    * Stemming: Reduces words to their root form.

    * Removing stopwords: Commonly occurring words like "and", "the", "is" that do not contribute significant meaning to the text are removed using a custom stopwords set.

    * Filtering empty texts after preprocessing.



In [2]:
import re
import nltk
import nlpaug.augmenter.word as naw
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

stopwords_set = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b0*\d+\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(word) for word in text.split() if word not in stopwords_set]
    return ' '.join(words) if words else 'empty'



2025-02-13 19:45:54.451948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739475954.474887   17097 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739475954.482019   17097 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 19:45:54.506251: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is a

In [3]:
data_train['title'] = data_train['title'].apply(clean_text)
data_test['title'] = data_test['title'].apply(clean_text)


Removing empty texts:

In [4]:
data_train = data_train[data_train['title'] != 'empty']
data_test = data_test[data_test['title'] != 'empty']

Before proceeding with the text classification tasks, it is important to highlight why we opted for a traditional classification algorithm rather than a deep learning approach. The primary reason is the relatively small size of our dataset.

Given that our dataset contains fewer than 10,000 labeled samples, deep learning models would struggle to generalize effectively and may overfit due to insufficient data. Specifically, text classification tasks typically require a large number of labeled examples to learn meaningful patterns without excessive overfitting.

For this reason, we chose **Support Vector Machines (SVM)**. SVMs are well-suited for small to medium-sized datasets and perform efficiently in high-dimensional spaces such as text classification. They also provide robust decision boundaries and handle class imbalances effectively by adjusting class weights.

# ST1

## Model Training

A Pipeline is used to streamline feature extraction and classification:

* TfidfVectorizer is applied to transform text into numerical features.

* SVC (Support Vector Classifier) is used for classification.

* GridSearchCV is employed to find optimal hyperparameters.

The model is trained separately for:

* hazard-category classification

* product-category classification


### Predict Hazard-Category

To optimize the model’s performance, we first applied GridSearchCV to determine the best hyperparameters for the Hazard-Category classification task.

In [5]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer


pipeline_hazard = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SVC(random_state=42, class_weight='balanced'))
])

param_grid = {
    'vect__ngram_range': [(1, 2), (1, 3)],
    'vect__max_features': [10000,20000,30000],
    'clf__C': [1, 10, 100],
    'clf__kernel': ['linear']  
}

# GridSearchCV
gs_clf = GridSearchCV(pipeline_hazard, param_grid, cv=3, n_jobs=-1)
gs_clf.fit(data_train['title'], data_train['hazard-category'])


best_params = gs_clf.best_params_
print(f" Best Parameters for Hazard-Category: {best_params}")

 Best Parameters for Hazard-Category: {'clf__C': 10, 'clf__kernel': 'linear', 'vect__max_features': 30000, 'vect__ngram_range': (1, 2)}


Using these values, we trained the final classifier for Hazard-Category.
Specifically:

* TfidfVectorizer: The ngram_range=(1, 2) parameter ensures that both unigrams (single words) and bigrams (pairs of consecutive words) are considered as features, capturing additional context. The max_features=30,000 limits the number of features to avoid excessive dimensionality, and sublinear_tf=True applies logarithmic scaling to term frequencies, improving robustness.

* Support Vector Machine (SVM): The C=10 parameter controls the regularization strength, where higher values reduce regularization and allow more flexibility in fitting the data. The kernel='linear' ensures a linear decision boundary, which is well-suited for text classification problems. The random_state=42 ensures reproducibility, and the class_weight='balanced' adjusts for imbalanced class distributions by assigning higher weights to underrepresented classes.

In addition to this, various parameters were tested across all models. For faster execution, some of these parameters were removed, and a more streamlined approach.

In [6]:
text_clf_hazard = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=best_params['vect__ngram_range'], max_features=best_params['vect__max_features'], sublinear_tf=True)),
    ('clf', SVC(C=best_params['clf__C'], kernel=best_params['clf__kernel'], random_state=42, class_weight='balanced'))
])

text_clf_hazard.fit(data_train['title'], data_train['hazard-category'])
predicted_hazard = text_clf_hazard.predict(data_test['title'])

print("Hazard Classification Report:")
print(metrics.classification_report(data_test['hazard-category'], predicted_hazard))

Hazard Classification Report:
                                precision    recall  f1-score   support

                     allergens       0.84      0.92      0.87       365
                    biological       0.90      0.91      0.90       343
                      chemical       0.88      0.69      0.77        52
food additives and flavourings       0.50      0.50      0.50         4
                foreign bodies       0.80      0.76      0.78       111
                         fraud       0.79      0.67      0.72        75
                     migration       0.00      0.00      0.00         1
          organoleptic aspects       0.67      0.40      0.50        10
                  other hazard       0.68      0.58      0.62        26
              packaging defect       0.60      0.60      0.60        10

                      accuracy                           0.85       997
                     macro avg       0.67      0.60      0.63       997
                  weighted avg  

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

### Predict Product-Category
Afterward, we repeated the GridSearchCV process for Product-Category to ensure the best possible performance for this task as well.

In [7]:
pipeline_product = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SVC(random_state=42, class_weight='balanced'))
])

param_grid = {
    'vect__ngram_range': [(1, 2), (1, 3)],
    'vect__max_features': [10000, 20000, 30000,],
    'clf__C': [1, 10, 100],
    'clf__kernel': ['linear']
}

gs_clf = GridSearchCV(pipeline_product, param_grid, cv=3, n_jobs=-1)
gs_clf.fit(data_train['title'], data_train['product-category'])

best_params = gs_clf.best_params_
print(f" Best Parameters for Product-Category: {best_params}")

 Best Parameters for Product-Category: {'clf__C': 10, 'clf__kernel': 'linear', 'vect__max_features': 30000, 'vect__ngram_range': (1, 2)}


With these optimized parameters, we proceed to train the final classifiers for Product-Category. Specifically:

* TfidfVectorizer: The ngram_range=(1, 2) parameter ensures that both unigrams (single words) and bigrams (pairs of consecutive words) are considered as features, capturing additional context. The max_features=30,000 limits the number of features to avoid excessive dimensionality, and sublinear_tf=True applies logarithmic scaling to term frequencies, improving robustness.

* Support Vector Machine (SVM): The C=10 parameter controls the regularization strength, where higher values reduce regularization and allow more flexibility in fitting the data. The kernel='linear' ensures a linear decision boundary, which is well-suited for text classification problems. The random_state=42 ensures reproducibility, and the class_weight='balanced' adjusts for imbalanced class distributions by assigning higher weights to underrepresented classes.

In [8]:
text_clf_product = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=best_params['vect__ngram_range'], max_features=best_params['vect__max_features'], sublinear_tf=True)),
    ('clf', SVC(C=best_params['clf__C'], kernel=best_params['clf__kernel'], random_state=42, class_weight='balanced'))
])

text_clf_product.fit(data_train['title'], data_train['product-category'])
predicted_product = text_clf_product.predict(data_test['title'])

print("Product Classification Report:")
print(metrics.classification_report(data_test['product-category'], predicted_product))

Product Classification Report:
                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.94      1.00      0.97        16
                      cereals and bakery products       0.61      0.74      0.67       121
     cocoa and cocoa preparations, coffee and tea       0.80      0.83      0.81        42
                                    confectionery       0.75      0.36      0.49        33
dietetic foods, food supplements, fortified foods       0.75      0.58      0.65        26
                                    fats and oils       1.00      0.67      0.80         6
                   food additives and flavourings       0.50      0.25      0.33         4
                            fruits and vegetables       0.69      0.74      0.71       103
                                 herbs and spices       0.71      0.85      0.77        20
                            honey and royal jelly       1.

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [9]:
def compute_score(hazards_true, products_true, hazards_pred, products_pred):
    f1_hazards = f1_score(hazards_true, hazards_pred, average='macro')
    f1_products = f1_score(products_true[hazards_pred == hazards_true], 
                           products_pred[hazards_pred == hazards_true], 
                           average='macro')
    return (f1_hazards + f1_products) / 2.

final_score = compute_score(data_test['hazard-category'], data_test['product-category'], 
                            predicted_hazard, predicted_product)

print(f"Score Sub-Task 1: {final_score:.3f}")

Score Sub-Task 1: 0.669


## ST2
### Predict Product

In [10]:
param_grid_prod = { 
    'vect__ngram_range': [(1, 2), (1, 3)],
    'vect__max_features': [10000,20000, 30000],
    'clf__C': [1, 10, 100],
    'clf__kernel': ['linear']
}

prod_pipeline = Pipeline([
    ('vect', TfidfVectorizer(sublinear_tf=True)),
    ('clf', SVC(random_state=42))
])

gs_prod = GridSearchCV(prod_pipeline, param_grid_prod, cv=3, n_jobs=-1)
gs_prod.fit(data_train['title'], data_train['product'])

best_params_prod = gs_prod.best_params_
print(f"Best Parameters for Product-Category: {best_params_prod}")


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best Parameters for Product-Category: {'clf__C': 10, 'clf__kernel': 'linear', 'vect__max_features': 20000, 'vect__ngram_range': (1, 2)}


The warning occurs because some classes in the target variable have fewer than 3 samples (either 0, 1, or 2), which is less than the number of splits (3) in the cross-validation. 

Using these values, we trained the final classifier for Product:

In [11]:
prod_text_clf = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=best_params_prod['vect__ngram_range'], 
                              max_features=best_params_prod['vect__max_features'], 
                              sublinear_tf=True)),
    ('clf', SVC(C=best_params_prod['clf__C'], kernel=best_params_prod['clf__kernel'], random_state=42, class_weight='balanced'))
])

prod_text_clf.fit(data_train['title'], data_train['product'])
predicted_prod = prod_text_clf.predict(data_test['title'])

print("Classification Report for Product Detection:")
print(metrics.classification_report(data_test['product'], predicted_prod))

Classification Report for Product Detection:
                                                               precision    recall  f1-score   support

                                       Catfishes (freshwater)       0.40      0.67      0.50         3
                                              Dried pork meat       0.00      0.00      0.00         1
                                        Fishes not identified       0.54      0.88      0.67         8
                                     Not classified pork meat       0.00      0.00      0.00         3
                          Precooked cooked pork meat products       0.00      0.00      0.00         1
                                            Saurida (generic)       0.00      0.00      0.00         1
                           Torpedo-shaped catfishes (generic)       0.00      0.00      0.00         0
                                                Veggie Burger       0.50      1.00      0.67         1
                           

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali

# Predict Hazard

In [12]:

param_grid_haz = { 
    'vect__ngram_range': [(1, 2), (1, 3)],
    'vect__max_features': [10000, 20000, 30000],
    'clf__C': [1, 10],
    'clf__kernel': ['linear']
}

haz_pipeline = Pipeline([
    ('vect', TfidfVectorizer(sublinear_tf=True)),
    ('clf', SVC(random_state=42))
])

gs_haz = GridSearchCV(haz_pipeline, param_grid_haz, cv=3, n_jobs=-1)
gs_haz.fit(data_train['title'], data_train['hazard'])

best_params_haz = gs_haz.best_params_
print(f"Best Parameters for Hazard-Category: {best_params_haz}")



Best Parameters for Hazard-Category: {'clf__C': 10, 'clf__kernel': 'linear', 'vect__max_features': 30000, 'vect__ngram_range': (1, 2)}


Using these values, we trained the final classifier for Hazard:

In [13]:

haz_text_clf = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=best_params_haz['vect__ngram_range'], 
                              max_features=best_params_haz['vect__max_features'], 
                              sublinear_tf=True)),
    ('clf', SVC(C=best_params_haz['clf__C'], kernel=best_params_haz['clf__kernel'], random_state=42, class_weight='balanced'))
])

haz_text_clf.fit(data_train['title'], data_train['hazard'])
predicted_haz = haz_text_clf.predict(data_test['title'])

print("Classification Report for Hazard Detection:")
print(metrics.classification_report(data_test['hazard'], predicted_haz))

Classification Report for Hazard Detection:
                                                 precision    recall  f1-score   support

                                      Aflatoxin       0.00      0.00      0.00         2
                                 abnormal smell       0.00      0.00      0.00         1
                                alcohol content       0.00      0.00      0.00         1
                                      alkaloids       1.00      1.00      1.00         1
                                      allergens       0.00      0.00      0.00         3
                                         almond       1.00      0.62      0.76        13
                                      amygdalin       0.00      0.00      0.00         1
                         antibiotics, vet drugs       1.00      1.00      1.00         1
                                  bacillus spp.       1.00      0.67      0.80         3
                           bad smell / off odor       0.00      0

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [14]:
def compute_score(hazards_true, products_true, hazards_pred, products_pred):
    f1_hazards = f1_score(hazards_true, hazards_pred, average='macro')
    f1_products = f1_score(products_true[hazards_pred == hazards_true], products_pred[hazards_pred == hazards_true], average='macro')
    
    return (f1_hazards + f1_products) / 2



final_score = compute_score(
    data_test['hazard'],      
    data_test['product'],      
    predicted_haz,             
    predicted_prod            
)

print(f"Score Sub-Task 2: {final_score:.3f}")


Score Sub-Task 2: 0.349


At this point, I am creating the submission file that includes the predictions for both product and hazard categories. The file will be saved as submission.csv in the submission folder, which will then be compressed into a zip file for submission as per the competition requirements:

In [15]:
import os
from shutil import make_archive

# save predictions to a new folder:
os.makedirs('./submission/', exist_ok=True)

submission_df = pd.DataFrame({
    'product-category': predicted_product,
    'hazard-category': predicted_hazard,
    'product': predicted_prod,
    'hazard': predicted_haz
})

submission_df.to_csv('./submission/submission.csv')

# zip the folder (zipfile can be directly uploaded to codalab):
make_archive('./submission', 'zip', './submission')

'/home/ec2-user/SageMaker/submission.zip'